In [3]:
def informacoes_auxiliares( 
    informacoes_card_imovel,
    cod_imoveis):
        
    qntd_items = []
    valores = []
    infos_extras_imovel = {}

    for j in range(0, len(informacoes_card_imovel)):
        infos_extras_imovel.setdefault(cod_imoveis[j], [])
    
        for k in range(0, len(informacoes_card_imovel[j])):
            # Salvando as variaveis em listas
            qntd_items.append( informacoes_card_imovel[j][k].img.next.text.strip() )
            valores.append( informacoes_card_imovel[j][k].img['src'][47:-4] )
        
            # Iterando e populando o dicionario
            infos_extras_imovel[cod_imoveis[j]].append(qntd_items)
            infos_extras_imovel[cod_imoveis[j]].append(valores)
        
            qntd_items = []
            valores = []
        
    df_aux = pd.DataFrame.from_dict(infos_extras_imovel, orient='index')
    return df_aux

def data_cleaning(
    card_iterator,
    codigo_imovel,
    local_imovel):
        
    #   print("passei 2.2")
        
    bairro = []
    cidade = []
    
    cod_imoveis = []
    
    for i in range(len(local_imovel)):
        bairro.append(local_imovel[i].split()[0])
        cidade.append(local_imovel[i].split()[1])
        
    for i in range(len(codigo_imovel)):
        cod_imoveis.append(codigo_imovel[i].split()[1])
    
    return bairro, cidade, cod_imoveis
    

In [13]:
# Base librarys
import bs4 as bs4
import urllib.request
import pandas as pd
import xlsxwriter

# AUX librarys
import requests
import time
import re

class Crawler_planeta():
    '''
    Crawler da Imobiliaria planeta, desenvolvido por
    @Carlosal249, o Crawler coleta todas as informacoes disponiveis no endpoint:
    'https://itaivan.com/busca/', e retorna elas em formato .xlsx, podendo ainda ser 
    enviada por E-mail.
    '''
    def __init__(self, qntd_paginas, lista_paginacao):
        self.qntd_paginas = qntd_paginas
        self.lista_paginacao = lista_paginacao
        
        self.headers = {
            "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36", 
            "Accept":"text/html,application/xhtml+xml,application/xml; q=0.9,image/webp,image/apng,*/*;q=0.8"
        }
        
        for i in range(1, qntd_paginas):
            self.lista_paginacao.append( r'https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=' + str(i) )
        
        print(" Objecto Instanciado com Sucesso!")
    
    def crawlear_(self):
        '''
        Funcao especifica para inciar o crawleamento dos dados
        '''
        data_final_01 = pd.DataFrame()
        data_final_02 = pd.DataFrame()
        
        for i in range(0, self.qntd_paginas-1):
            url = self.lista_paginacao[i]
            
            source = urllib.request.urlopen(url).read()
            time.sleep(10)

            soup = bs4.BeautifulSoup(source,'html')
            
            # Instanciando objetos Base da busca
            cards_imovel = soup.find_all('div', class_='slide col-xs-12 col-sm-6 col-md-6 col-lg-4')
            informacoes_card_imovel = [card.find_all('span', class_='tooltip-control') for card in cards_imovel]
    
            teste_imagem = [card.find('div', class_='ls-picture').get('style') for card in cards_imovel]
            imagens_tratadadas = [re.sub('background-image: url', '', imagem) for imagem in teste_imagem]
            tipo_imovel = [card.find('div', class_='ls-categoria').get_text().strip() for card in cards_imovel]
            local_imovel = [card.find('span', class_='ls-loc block').get_text().strip().replace(r' ','') for card in cards_imovel]
            codigo_imovel = [card.find('span', class_='ls-cod').get_text() for card in cards_imovel]
            try:
                m2_imovel = [card.find('span', class_='tooltip-control').get_text().strip() for card in cards_imovel]
                
            except:
                m2_imovel = [0, 0, 0, 0, 0, 0, 0, 0, 0]
                
            try:
                preco_imovel = [card.find('span', class_='block ls-price color').get_text().strip() for card in cards_imovel]
                
            except:
                preco_imovel = [0, 0, 0, 0, 0, 0, 0, 0, 0]
            #print(preco_imovel)
            
            print(f'Passando pela Pagina {self.lista_paginacao[i]}, iteracao N:{i}')
            
            bairro, cidade, cod_imoveis = data_cleaning(
                card_iterator=local_imovel,
                codigo_imovel=codigo_imovel,
                local_imovel=local_imovel
            )
            
            df_aux  = informacoes_auxiliares(
                informacoes_card_imovel = informacoes_card_imovel,
                cod_imoveis =  cod_imoveis
            )
            
            data = pd.DataFrame(list(
                zip(tipo_imovel, bairro, cidade, 
                    cod_imoveis, m2_imovel,
                    preco_imovel,imagens_tratadadas)),
                columns=[
                    'Tipo_imovel', 'bairro', 'cidade',
                    'codigo_imovel', 'm2_imovel',
                    'preco_imovel', 'imagem_imovel'
                ], index=cod_imoveis)
            
            print('')
            
            data_final_01 = pd.concat([data_final_01, data])
            data_final_02 = pd.concat([data_final_02, df_aux])
        
        return data_final_01, data_final_02

In [16]:
lista_pag = []
crawler_head = Crawler_planeta(qntd_paginas=110, lista_paginacao=lista_pag)

df_1, df_2 = crawler_head.crawlear_()

 Objecto Instanciado com Sucesso!
Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=1, iteracao N:0

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=2, iteracao N:1

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=3, iteracao N:2

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=4, iteracao N:3

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=5, iteracao N:4

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=6, iteracao N:5

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=7, iteracao N:6

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=66, iteracao N:65

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=67, iteracao N:66

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=68, iteracao N:67

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=69, iteracao N:68

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=70, iteracao N:69

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=71, iteracao N:70

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=72, iteracao N:71

Passando pela Pagina https://itaivan.com/busca/?finalidade=Venda&valor%5B0%5D=&valor%5B1%5D=&codigo=&pagina=73,

In [26]:
with pd.ExcelWriter('BASE_COMPLETA_ITAIVAN_21_07.xlsx', engine='xlsxwriter') as writer:
    # Saving both Dataframes in the same xlsx file
    df_1.to_excel(writer, sheet_name='Imoveis')
    df_2.to_excel(writer, sheet_name='Complementos')
    

In [14]:
df1.to_csv('Integracao_pbi_19_07.csv')
df2.to_csv('Integracao_aux_pbi_19_07.csv')

In [35]:
df1['imagem_imovel'][0]

'https://cdn.vistahost.com.br/itaivan1/vista.imobi/fotos/12363/i3511S3K57mnE04r51wrX_1236362d6e58d41b9a.jpg'

In [9]:
df1.to_csv("INFOS_GERAIS_IMOVEIS_18_07_22.csv")
df2.to_csv("INFOS_DETALHADAS_IMOVEIS_18_07_22.csv")

In [27]:
df1['imagem_imovel'][0]
# Todo's, Vba script to automatic render the image on the xlsx file

'https://cdn.vistahost.com.br/itaivan1/vista.imobi/fotos/12363/i3511S3K57mnE04r51wrX_1236362d6e58d41b9a.jpg)'